# Sistema de recomendación de anime

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 
from sklearn.neighbors import NearestNeighbors

#Barra de progreso
import time
from tqdm import tqdm


El objetivo es aplicar todo lo que hemos aprendido hasta el momento, especialmente sobre gradiente descendiente. Para esto desarrollaremos un sistema de recomendación usando factorización de matrices.

En esta entrega se darán todos elementos necesarios para desarrollar un sistema de recomendación de amines, el objetivo es que usted entienda como funciona y desarrolle una solución.

Si desea conocer mas sobre el dataset que usaremos puede revisar acá: https://www.kaggle.com/CooperUnion/anime-recommendations-database

1- Cargue las bases de datos de calificaciones, usuarios y animes. Puede usar para este objetivo la biblioteca pandas. Genere el conjunto de entrenamiento usando los usuarios cómo filas y los animes cómo columnas. Tenga en cuenta que para el algoritmo es necesario que los -1 sean cero, sin embargo, para la recomendación si debe tener en cuenta dichos valores.

In [2]:
#Obtener Listado .CSV
def cargarAnimes():
    df = pd.read_csv('anime.csv', delimiter=',', parse_dates=[6], header=0,index_col=False, squeeze=False)
    return np.array(df)

#Números globales cantidad animes y usuarios
cantAnimes=12294
cantUsers=73516

#Crear R a partir del archivo csv
def crearR(ceros):
    aux=-1
    if ceros:
        aux=0
    uf = pd.read_csv('rating.csv', delimiter=',', parse_dates=[2], header=0,index_col=False, squeeze=False)
    u=np.array(uf)
    
    #Crear R
    R = np.zeros((cantUsers,cantAnimes), dtype = np.int8)
    for i in u:
        if i[1] <= cantAnimes and int(i[2])>=aux:
            R[i[0]-1,i[1]-1] = int(i[2])
      
    return R 


2- Agregue la función donde se ejecuta el algoritmo de factorización de matrices. Puede basarse en el algoritmo compartido en las dispositivas. No debe de gastar todas las iteraciones. 

In [3]:
def matrix_factorization(R, P, Q, K, steps=500, alpha=0.01, beta=0.001):
    Q = Q.T
    error=0
    for step in tqdm(range(steps)):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        aux1=(2 * eij * Q[k][j] - beta * P[i][k])
                        aux2=(2 * eij * P[i][k] - beta * Q[k][j])
                        P[i][k] = P[i][k] + alpha * aux1
                        Q[k][j] = Q[k][j] + alpha * aux2
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        error=e
        if e < 0.5:
            break
    print('Error',error)
    return P, Q.T

3- Programe una función para generar $\hat{R}$. Recuerde que para generar $\hat{R}$ debe realizar los siguiente pasos:

    - Generar R usando el algoritmo de vecinos más cercanos.
    - Defina K.
    - Defina P y Q.
    - Generar R prima.


In [4]:
def guardar(name,matriz):
    np.savetxt(name,matriz)

def cargar(name):
    return np.loadtxt(name)

def generateRandomPQ(k):
    Q = np.random.random((cantAnimes, k)).astype(np.float32)
    P = np.random.random((cantUsers, k)).astype(np.float32)
    return P,Q

def generateR(usuario, vecinos):
    R = crearR(True)
    RSelected = np.empty([vecinos,cantAnimes])
    distances,indices=knn(True,usuario,vecinos,R)
    for i in range(0, len(indices[0])-1, 1):
        RSelected[i] = R[indices[0,i]]
    del R
    return RSelected

def knn(cargado,usuario,vecinos,R):
    if not(cargado):
        nbrs = NearestNeighbors(n_neighbors=vecinos, algorithm='auto', n_jobs=-1).fit(R)
        distances, indices = nbrs.kneighbors(R[usuario-1].reshape((1, -1)))
        return distances,indices
    else:
        if vecinos>30:
            vecinos=30
        if usuario in [51,510,17000,27005,37502,57502,67501]:
            distances=np.loadtxt('distances'+str(usuario)+'.txt')
            indices=np.loadtxt('indices'+str(usuario)+'.txt')
            return distances[:vecinos].reshape(1,vecinos),indices[:vecinos].reshape(1,vecinos).astype(int)
        else:
            distances=np.loadtxt('distances.txt')
            indices=np.loadtxt('indices.txt')
            return distances.reshape(1,vecinos),indices.reshape(1,vecinos).astype(int)
        
def optimizarMatriz(R,indices,distances):
    userIguales=np.array([]).reshape(0,2)
    cont=0
    pos=-1
    Raux=np.zeros((len(R),len(R[0])),dtype=np.int8)
    for i in tqdm(range(len(indices))):
        for j in range(len(indices[0])):
            if int(distances[i,j]) == 0:
                index=int(indices[i,j])
                aux1=np.where(userIguales[:,1]==i)
                if i != index:
                    aux2=np.where(userIguales[:,1]==index)
                    if len(aux1[0]) == 0:
                        if len(aux2[0]) == 0:
                            pos=pos+1
                            userIguales=np.insert(userIguales, userIguales.shape[0], np.array([pos,i]), 0)
                            userIguales=np.insert(userIguales, userIguales.shape[0], np.array([pos,index]), 0)
                            Raux[cont]=R[i]
                            cont=cont+1
                        else:
                            userIguales=np.insert(userIguales,aux2[0][-1]+1, np.array([userIguales[aux2[0][-1],0],i]), 0)
                    else:
                        if len(aux2[0]) == 0:
                            userIguales=np.insert(userIguales,aux1[0][-1]+1, np.array([userIguales[aux1[0][-1],0],index]), 0)                  
                else:
                    if len(aux1[0]) == 0:
                        pos=pos+1
                        userIguales=np.insert(userIguales, userIguales.shape[0], np.array([pos,i]), 0)
                        Raux[cont]=R[i]
                        cont=cont+1
    return Raux[:cont,:], userIguales

def generateRPrime(k, usuario, vecinos):
    P,Q = generateRandomPQ(k)
    RSelected = generateR(usuario, vecinos)
    P,Q = matrix_factorization(RSelected, P, Q, k)
    return RSelected, np.dot(P, Q.T)

def obtenerRPrima(cargado,usuario,kNeighbors,kFactorizacion):
    if not(cargado) or not(usuario in [51,510,17000,27005,37502,57502,67501]):
        Rl, Rp = generateRPrime(kFactorizacion, usuario, kNeighbors)
        return Rl,Rp
    else:
        Rl=crearR(False)
        Rp=cargar('Rp'+str(usuario)+'V20.txt').reshape(12294,1).T
        return Rl,Rp

4- Realice las siguiente recomendaciones (animes que no ha visto con calificación 10):

    1- Usuario de id 51, películas.
    2- Usuario de id 510, animes con menos de 31 episodios.
    3- Usuario de id 17000, animes con rankins mayores o iguales a 9.
    4- Usuario de id 27005, animes de comedia y aventura.
    5- Usuario de id 37502, TV.
    6- Usuario de id 57502, películas y OVAS.
    7- Usuario de id 67501.

Cada recomendación debe tener como respuesta, número de iteraciones, número de recomendaciones y nombre de los animes,

In [45]:
def recomendar(RpSelected,RSelected,d,idUser,minRating=10,genre=[],tipo=[],episodios=None,ranking=None,members=None):  
    recomendacion=np.array([],dtype='object').reshape(0,len(d[0]))  
    for i in tqdm(range(len(d))):
        if RpSelected[i]>=minRating and RSelected[0,i]!=-1:
            boolean=True
            if genre!=None and len(genre)!=0:
                ward=False
                for j in genre:
                    if j in d[i,2]:
                        ward=True
                if not(ward):
                    boolean=False  
            if boolean and len(tipo)!=0:
                ward=False
                for j in tipo:
                    if j in d[i,3]:
                        ward=True
                if not(ward):
                    boolean=False  
            if boolean and episodios!=None:
                try:
                    if not(episodios >= int(d[i,4])):
                        boolean=False
                except ValueError:
                        boolean=False
            if boolean and ranking!=None:
                try:
                    if not(ranking <= int(d[i,5])):
                        boolean=False
                except ValueError:
                        boolean=False  
            if boolean and members!=None:
                try:
                    if not(members <= int(d[i,5])):
                        boolean=False
                except ValueError:
                        boolean=False 
            if boolean:
                recomendacion=np.insert(recomendacion, recomendacion.shape[0], d[i], 0)                
    return recomendacion

def imprimirRecomendacion(recomendacion):
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    rd = {'id' : pd.Series(recomendacion[:,0].T),'nombre' : pd.Series(recomendacion[:,1].T),'Genero' : pd.Series(recomendacion[:,2].T),'tipo' : pd.Series(recomendacion[:,3].T),'episodios' : pd.Series(recomendacion[:,4].T),'rating' : pd.Series(recomendacion[:,5].T),'miembros' : pd.Series(recomendacion[:,6].T)}
    pd.DataFrame(rd)

In [ ]:
def imprimirDiferencias(idUser,Rl,Rp,minR=0,minRp=0,maxR=10,maxRp=100):
    pos=0
    a=idUser-1
    b=0
    for i in range(len(Rl[0])-1):
        if Rl[a,i] != np.around(Rp[b,i]) and (Rl[a,i]>=minR and Rp[b,i]>=minRp and Rl[a,i]<=maxR and Rp[b,i]<=maxRp):
                print('UserAntes',Rl[a,i],'UserDespues',Rp[b,i])
                pos=pos+1
    print('Cantidad Diferencias: ',pos)
    
print(Rp)
imprimirDiferencias(67501,Rl,Rp,minR=1)

In [ ]:
#HALLAR R PRIMA

usuario=51
kNeighbors=20
kFactorizacion=20

#Usuario
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)

print(Rp)
print(Rl)

In [26]:
#Establecer Parametros

usuario=51
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10,tipo=['Movie'])

print(recomendacion)
pd.set_option("display.max_rows", None, "display.max_columns", None)
rd = {'id' : pd.Series(recomendacion[:,0].T),'nombre' : pd.Series(recomendacion[:,1].T),'Genero' : pd.Series(recomendacion[:,2].T),'tipo' : pd.Series(recomendacion[:,3].T),'episodios' : pd.Series(recomendacion[:,4].T),'rating' : pd.Series(recomendacion[:,5].T),'miembros' : pd.Series(recomendacion[:,6].T)}
pd.DataFrame(rd)
#imprimirRecomendacion(recomendacion)
del Rl
del Rp

100%|████████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 948247.92it/s]

[[32281 'Kimi no Na wa.' 'Drama, Romance, School, Supernatural' 'Movie'
  '1' 9.37 '200630']
 [12355 'Ookami Kodomo no Ame to Yuki' 'Fantasy, Slice of Life' 'Movie'
  '1' 8.84 '226193']
 [12859 'One Piece Film: Z'
  'Action, Adventure, Comedy, Drama, Fantasy, Shounen' 'Movie' '1' 8.39
  '76051']
 [6467 'Detective Conan Movie 14: The Lost Ship in the Sky'
  'Action, Mystery, Police, Shounen' 'Movie' '1' 8.29 '20602']
 [21469 'Stand By Me Doraemon' 'Comedy, Kids, Sci-Fi, Shounen' 'Movie'
  '1' 8.12 '5712']
 [10717 'Towa no Quon 6: Towa no Quon'
  'Action, Super Power, Supernatural' 'Movie' '1' 7.79 '18624']
 [3785 'Evangelion: 3.0 You Can (Not) Redo' 'Action, Mecha, Sci-Fi'
  'Movie' '1' 7.71 '135318']
 [14669 'Aura: Maryuuin Kouga Saigo no Tatakai'
  'Comedy, Drama, Romance, School, Supernatural' 'Movie' '1' 7.67 '22599']
 [2656 'Doraemon Movie 25: Nobita no Wan Nyan Jikuuden'
  'Adventure, Comedy, Kids, Sci-Fi' 'Movie' '1' 7.47 '1269']
 [2427 'Unico' 'Adventure, Fantasy, Kids' 'Movie' 

In [7]:
#Establecer Parametros

usuario=510
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10,episodios=31)

print(recomendacion)
pd.set_option("display.max_rows", None, "display.max_columns", None)
rd = {'id' : pd.Series(recomendacion[:,0].T),'nombre' : pd.Series(recomendacion[:,1].T),'Genero' : pd.Series(recomendacion[:,2].T),'tipo' : pd.Series(recomendacion[:,3].T),'episodios' : pd.Series(recomendacion[:,4].T),'rating' : pd.Series(recomendacion[:,5].T),'miembros' : pd.Series(recomendacion[:,6].T)}
pd.DataFrame(rd)
#imprimirRecomendacion(recomendacion)
del Rl
del Rp

100%|████████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 880485.85it/s]

[[7311 'Suzumiya Haruhi no Shoushitsu'
  'Comedy, Mystery, Romance, School, Sci-Fi, Supernatural' 'Movie' '1'
  8.81 '240297']
 [12859 'One Piece Film: Z'
  'Action, Adventure, Comedy, Drama, Fantasy, Shounen' 'Movie' '1' 8.39
  '76051']
 [2167 'Clannad'
  'Comedy, Drama, Romance, School, Slice of Life, Supernatural' 'TV' '23'
  8.3 '566690']
 [5365 'Tsumiki no Ie' 'Drama' 'Movie' '1' 8.27 '45189']
 [4938 'Tsubasa: Shunraiki'
  'Action, Adventure, Drama, Fantasy, Magic, Mystery, Romance, Shounen, Supernatural'
  'OVA' '2' 8.23 '40420']
 [934 'Higurashi no Naku Koro ni'
  'Horror, Mystery, Psychological, Supernatural, Thriller' 'TV' '26' 8.17
  '359494']
 [6505 'There She Is!!' 'Comedy, Romance' 'ONA' '5' 8.11 '13935']
 [779 'Detective Conan Movie 01: The Timed Skyscraper'
  'Adventure, Comedy, Mystery, Police, Shounen' 'Movie' '1' 7.98 '28947']
 [875 'Mind Game' 'Adventure, Comedy, Dementia, Psychological, Romance'
  'Movie' '1' 7.88 '32756']
 [9074 'Arakawa Under the Bridge x Bridge' 

In [47]:
#Establecer Parametros

usuario=17000
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10,ranking=8)
print(recomendacion)
imprimirRecomendacion(recomendacion)
del Rl
del Rp

100%|███████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 1123268.71it/s]

[[19 'Monster'
  'Drama, Horror, Mystery, Police, Psychological, Seinen, Thriller' 'TV'
  '74' 8.72 '247562']
 [7785 'Yojouhan Shinwa Taikei' 'Mystery, Psychological, Romance' 'TV'
  '11' 8.65 '122531']
 [12859 'One Piece Film: Z'
  'Action, Adventure, Comedy, Drama, Fantasy, Shounen' 'Movie' '1' 8.39
  '76051']
 [23327 'Space☆Dandy 2nd Season' 'Comedy, Sci-Fi, Space' 'TV' '13' 8.27
  '60918']
 [268 'Golden Boy' 'Adventure, Comedy, Ecchi' 'OVA' '6' 8.05 '113040']
 [29093 'Grisaia no Meikyuu: Caprice no Mayu 0' 'Drama' 'Special' '1'
  8.05 '70039']
 [32660 'Petit Petit Muse' 'Cars, Horror, Kids' 'TV' '26' 8.14 '78']]


In [46]:
#Establecer Parametros

usuario=27005
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10,genre=['Comedy','Adventure'])
print(recomendacion)
imprimirRecomendacion(recomendacion)
del Rl
del Rp

100%|████████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 821827.96it/s]

[[2001 'Tengen Toppa Gurren Lagann'
  'Action, Adventure, Comedy, Mecha, Sci-Fi' 'TV' '27' 8.78 '562962']
 [24415 'Kuroko no Basket 3rd Season' 'Comedy, School, Shounen, Sports'
  'TV' '25' 8.62 '184525']
 [4224 'Toradora!' 'Comedy, Romance, School, Slice of Life' 'TV' '25'
  8.45 '633817']
 [137 'Hunter x Hunter OVA' 'Action, Adventure, Shounen, Super Power'
  'OVA' '8' 8.41 '53168']
 [1210 'NHK ni Youkoso!' 'Comedy, Drama, Psychological, Romance' 'TV'
  '24' 8.4 '291228']
 [3901 'Baccano! Specials'
  'Action, Comedy, Historical, Mystery, Seinen, Supernatural' 'Special'
  '3' 8.29 '100412']
 [10643 'Gintama: Dai Hanseikai' 'Action, Comedy, Parody, Samurai'
  'Special' '1' 8.19 '14728']
 [232 'Cardcaptor Sakura'
  'Adventure, Comedy, Drama, Fantasy, Magic, Romance, School, Shoujo'
  'TV' '70' 8.18 '181249']
 [6421 'Fullmetal Alchemist: Brotherhood Specials'
  'Adventure, Drama, Fantasy, Magic, Military, Shounen' 'Special' '4'
  8.11 '67962']
 [29803 'Overlord'
  'Action, Adventure, Fan

In [25]:
#Establecer Parametros

usuario=37502
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10,tipo=['TV'])
print(recomendacion)
imprimirRecomendacion(recomendacion)
del Rl
del Rp

100%|███████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 1224991.05it/s]

[[396 'Seikai no Senki' 'Action, Military, Romance, Sci-Fi, Space' 'TV'
  '13' 7.86 '18580']
 [13409 'Moyashimon Returns' 'Comedy, School, Supernatural' 'TV' '11'
  7.25 '16251']
 [15883 'Fantasista Doll' 'Fantasy, Magic, Sci-Fi' 'TV' '12' 6.24 '18799']]


In [24]:
#Establecer Parametros

usuario=57502
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10,tipo=['OVA','Movie'])
print(recomendacion)
imprimirRecomendacion(recomendacion)
del Rl
del Rp

100%|████████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 425080.16it/s]

[[30346 'Doukyuusei (Movie)' 'Romance, School, Shounen Ai, Slice of Life'
  ... '1' 8.53 '28864']
 [4155 'One Piece Film: Strong World'
  'Action, Adventure, Comedy, Drama, Fantasy, Shounen' ... '1' 8.42
  '85020']
 [9130 'Saint Seiya: The Lost Canvas - Meiou Shinwa 2'
  'Action, Adventure, Martial Arts, Shounen, Super Power, Supernatural'
  ... '13' 8.36 '27532']
 ...
 [4178 'Dream Hazard: Akuma no Program' 'Hentai' ... '1' 4.77 '616']
 [19747 'Dakaretai Onna' 'Hentai, Romance' ... '2' 3.43 '144']
 [9504 'Sakura no Mori' 'Hentai' ... '2' 4.53 '221']]


In [27]:
#Establecer Parametros

usuario=67501
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10)
print(recomendacion)
imprimirRecomendacion(recomendacion)
del Rl
del Rp

100%|███████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 1118808.68it/s]

[[19195 'Ghost in the Shell: Arise - Border:4 Ghost Stands Alone'
  'Mecha, Police, Psychological, Sci-Fi' 'Movie' '1' 7.6 '19565']
 [23311 'Garo: Honoo no Kokuin'
  'Action, Demons, Fantasy, Magic, Supernatural' 'TV' '24' 7.51 '64214']
 [17535 'Fairy Tail Movie 1: Houou no Miko - Hajimari no Asa'
  'Fantasy, Magic, Shounen' 'Special' '1' 7.49 '34230']
 [10444 'Digimon Xros Wars: Aku no Death General to Nanatsu no Oukoku'
  'Action, Adventure, Comedy, Fantasy, Shounen' 'TV' '24' 7.42 '12505']
 [1678 'Cyborg 009: The Cyborg Soldier'
  'Action, Adventure, Mecha, Sci-Fi, Shounen' 'TV' '52' 7.36 '14867']
 [2116 'Captain Tsubasa' 'Action, Shounen, Sports' 'TV' '128' 7.35
  '36623']
 [526 'Boku no Chikyuu wo Mamotte' 'Drama, Sci-Fi, Shoujo' 'OVA' '6' 7.29
  '7254']
 [12695 'Tight-rope' 'Romance, Shounen Ai' 'OVA' '2' 7.28 '18866']
 [5688 'Gegege no Kitarou (1968)'
  'Adventure, Comedy, Fantasy, Horror, Supernatural' 'TV' '65' 7.06
  '1276']
 [670 'Lamune' 'Drama, Romance, Slice of Life' 'TV'